In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import math
from torch.nn.utils import prune
import tqdm.notebook as tqdm
from functools import partial
import torchvision
from collections import OrderedDict
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline

def set_global_seed(seed: int) -> None:
    """
    Set global seed for reproducibility.
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_global_seed(42)

In [ ]:
BATCH_SIZE = 128
DEVICE = torch.device('cpu')
if torch.cuda.is_available():
    DEVICE = torch.device('cuda', 0)

print(type(DEVICE), DEVICE)

## Загрузка и обработка данных

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomCrop(32, 4),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
ds_train = torchvision.datasets.CIFAR10(
    root='./', train=True, transform=transform, download=True
)
ds_test = torchvision.datasets.CIFAR10(
    root='./', train=False,
    transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )]),
    download=True
)
ds_train, ds_test

In [ ]:
tmean, tstd = transform.transforms[-1].mean, transform.transforms[-1].std
tmean, tstd = np.array(tmean), np.array(tstd)

inverse_transform = torchvision.transforms.Compose([
    torchvision.transforms.Normalize(
        mean=-tmean / tstd,
        std=1.0 / tstd
    ),
    torchvision.transforms.ToPILImage()
])

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(13, 6))

for idx, ds in enumerate((ds_train, ds_test)):
    for jdx, kdx in enumerate(np.random.randint(0, len(ds), size=5)):
        image, label = ds[kdx]
        axes[idx, jdx].imshow(inverse_transform(image))
        axes[idx, jdx].set_title(f'Метка: {label} -> {ds.classes[label]}')

axes[0, 0].set_ylabel('Обучающая выборка')
axes[1, 0].set_ylabel('Тестовая выборка')

fig.tight_layout()
plt.show()

In [ ]:
dl_train = torch.utils.data.DataLoader(
    dataset=ds_train, batch_size=BATCH_SIZE,
    num_workers=2, shuffle=True
)
dl_test = torch.utils.data.DataLoader(
    dataset=ds_test, batch_size=BATCH_SIZE,
    num_workers=2, shuffle=False
)

In [ ]:
class ConvNet(torch.nn.Module):
    cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

    def __init__(self, n_classes=10, use_batchnorm=False, dropout_p=0.0):
        '''
        :param int n_classes: Число выходных признаков
        :param bool use_batchnorm: Использовать ли батчнорм между свёрточными слоями
        :param float dropout_p: Вероятность обнуления активации слоем Dropout
        '''
        super().__init__()

        self.n_classes = n_classes

        self.in_channels = 3
        self.features = torch.nn.Sequential()
        for cfg_item in self.cfg:
            if isinstance(cfg_item, int):
                self.features.append(torch.nn.Conv2d(self.in_channels,
                                                     out_channels=int(cfg_item),
                                                     kernel_size=3,
                                                     padding=1))
                if use_batchnorm:
                    self.features.append(torch.nn.BatchNorm2d(int(cfg_item)))
                self.features.append(torch.nn.ReLU(inplace=True))
                self.in_channels = int(cfg_item)
            elif cfg_item == "M":
                self.features.append(torch.nn.MaxPool2d(kernel_size=(2, 2), stride=2))

        # self.avgpool = torch.nn.AdaptiveAvgPool2d(output_size=(2, 2))
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(in_features=512, out_features=512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=dropout_p),
            torch.nn.Linear(in_features=512, out_features=512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=dropout_p),
            torch.nn.Linear(in_features=512, out_features=10)
        )
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        # x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
conv_net = ConvNet()
conv_net.load_state_dict(torch.load(GLOBAL_PATH + 'Models/VGG_09.11.2022-16:50.pth', map_location=DEVICE))

In [ ]:
def test_model(model, train_dataloader) -> float:
    model.to(DEVICE)
    model.eval()
    with torch.no_grad():
        train_accuracies = []
        for images, labels in tqdm.tqdm(train_dataloader, total=len(train_dataloader), leave=False):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs  = model(images)
            train_accuracies.append(torch.sum(outputs.argmax(dim=1) == labels)/labels.shape[0])
        return float((sum(train_accuracies) / len(train_accuracies)).cpu() * 100)

In [ ]:
test_model(conv_net, dl_test)

In [ ]:
def count_parameters(net) -> int:
  count_params = 0
  count_zero_params = 0
  for name, module in net.named_modules():
      if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
          count_params += module.weight.numel()
          count_zero_params += torch.sum(module.weight == 0)
          sparsity = 100. * float(torch.sum(module.weight == 0)) / float(module.weight.nelement())
          print(f'Sparsity in {name}.weight with {module.weight.numel()} parameters: {sparsity:0.3f}%')
  print(f'Global sparsity: {100. * (float(count_zero_params) / float(count_params)):0.3f}%')
  return count_params

In [ ]:
count_parameters(conv_net)

## L1 norm prune unstructured




In [ ]:
conv_net = ConvNet()
conv_net.load_state_dict(torch.load(GLOBAL_PATH + 'Models/VGG_09.11.2022-16:50.pth', map_location=DEVICE))

In [ ]:
def prune_l1_unstructured(net, amount, dim):
    module_norm_list = {}
    for name, module in conv_net.named_modules():
          if isinstance(module, torch.nn.Conv2d):
              module_norm = torch.linalg.norm(module.weight.flatten(start_dim=1), ord=1, dim=1).cpu().detach().numpy()
              prune.ln_structured(module, name='weight', amount=amount, n=1, dim=dim)
              print(module.weight.shape)
              cur_module_df = pd.DataFrame({'L1 норма': module_norm,
                                            'Фильтры': module_norm < np.quantile(module_norm, amount)
                                            })
              print(max(module_norm[module_norm < np.quantile(module_norm, amount)]))
              cur_module_df.loc[cur_module_df['Фильтры'] == True, 'Фильтры'] = 'Исключаются'
              cur_module_df.loc[cur_module_df['Фильтры'] == False, 'Фильтры'] = 'Сохраняются'
              fig = px.bar(cur_module_df, x=cur_module_df.index, y='L1 норма',
                           color='Фильтры', color_discrete_map={'Исключаются': '#3192C1', 'Сохраняются': '#65C0CB'},
                           labels={'index': 'Номер фильтра'})
              fig.update_layout(
                 font=dict(
                    size=18,
                )
              )
              fig.show()
              module_norm_list[name] = module_norm

In [ ]:
prune_l1_unstructured(conv_net, 0.2, 0)

In [ ]:
count_parameters(conv_net)

In [ ]:
torch.all(conv_net.features[0].weight.flatten(1) == 0.0, dim=1)

In [ ]:
test_model(conv_net, dl_test)

In [ ]:
def apply_mask(net, buffer):
    conv_layer_num = 0
    for name, module in conv_net.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.custom_from_mask(module=module, name='weight', mask=buffer[conv_layer_num])
            conv_layer_num += 1
    torch.cuda.empty_cache()

In [ ]:
apply_mask(conv_net, list(conv_net.buffers()))

In [ ]:
def training_loop(n_epochs, network, loss_fn, optimizer,scheduler, dl_train, dl_test, device):
    '''
    :param int n_epochs: Число итераций оптимизации
    :param torch.nn.Module network: Нейронная сеть
    :param Callable loss_fn: Функция потерь
    :param torch.nn.Optimizer optimizer: Оптимизатор
    :param torch.utils.data.DataLoader dl_train: Даталоадер для обучающей выборки
    :param torch.utils.data.DataLoader dl_test: Даталоадер для тестовой выборки
    :param torch.Device device: Устройство на котором будут происходить вычисления
    :returns: Списки значений функции потерь и точности на обучающей и тестовой выборках после каждой итерации
    '''
    buffer = list(network.buffers())
    loss_fn.to(device)
    train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []
    pbar = tqdm.tqdm(range(n_epochs), total=n_epochs, leave=False)
    for epoch in (pbar):
        network.train()

        # Итерация обучения сети
        tmp_train_losses, tmp_train_accuracies = [], []
        for images, labels in tqdm.tqdm(dl_train, total=len(dl_train), leave=False):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs  = network(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            apply_mask(network, buffer)

            tmp_train_losses.append(loss)
            tmp_train_accuracies.append(torch.sum(outputs.argmax(dim=1) == labels)/labels.shape[0])

        train_losses.append((sum(tmp_train_losses) / len(tmp_train_losses)).cpu())
        train_accuracies.append((sum(tmp_train_accuracies) / len(tmp_train_accuracies)).cpu() * 100)
        # Оцениваем качество модели каждые 3 итерации
        if epoch % 1 == 0 or epoch == n_epochs - 1:
            # Переводим сеть в инференс режим
            network.eval()

            # При тестировании сети нет необходимости считать градиенты, поэтому можно отключить автоматическое дифференцирование
            #   для ускорения операций
            with torch.no_grad():
                # Вычисление качества и функции потерь на тестовой выборке
                tmp_test_losses, tmp_test_accuracies = [], []
                for images, labels in tqdm.tqdm(dl_test, total=len(dl_test), leave=False):
                    images = images.to(device)
                    labels = labels.to(device)

                    outputs  = network(images)

                    tmp_test_losses.append(loss_fn(outputs, labels))
                    tmp_test_accuracies.append(torch.sum(outputs.argmax(dim=1) == labels)/labels.shape[0])

                test_losses.append((sum(tmp_test_losses) / len(tmp_test_losses)).cpu())
                test_accuracies.append((sum(tmp_test_accuracies) / len(tmp_test_accuracies)).cpu() * 100)

            pbar.set_description(
                'Loss (Train/Test): {0:.3f}/{1:.3f}. Accuracy, % (Train/Test): {2:.2f}/{3:.2f}\n'.format(
                    train_losses[-1], test_losses[-1], train_accuracies[-1], test_accuracies[-1]
                )
            )

    return train_losses, test_losses, train_accuracies, test_accuracies

## 20% L1 norm prune unstructured


In [ ]:
conv_net = ConvNet()
conv_net.load_state_dict(torch.load(GLOBAL_PATH + 'Models/VGG_09.11.2022-16:50.pth', map_location=DEVICE))

In [ ]:
prune_l1_unstructured(conv_net, 0.2, 0)

In [ ]:
count_parameters(conv_net)

In [ ]:
test_model(conv_net, dl_test)

In [ ]:
train_func = partial(
    training_loop, n_epochs=2, loss_fn=torch.nn.CrossEntropyLoss(),
    dl_train=dl_train, dl_test=dl_test, device=DEVICE
)

In [ ]:
conv_net.to(DEVICE);

In [ ]:
optimizer = torch.optim.SGD(conv_net.parameters(), lr=0.05 * 2e-2, momentum=0.9, weight_decay=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.9)

In [ ]:
train_losses, test_losses, train_accs, test_accs = train_func(
    network=conv_net,
    optimizer=optimizer,
    scheduler=scheduler
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(train_accs, label="Точность на обучении", color='red', marker='.', linestyle='-.')
ax.plot(test_accs, label="Точность на тесте", color='red', marker='*')

ax.set_xlabel("Номер эпохи")
ax.set_ylabel("$\%$")

ax.grid(True)
ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
count_parameters(conv_net)

In [ ]:
test_model(conv_net, dl_test)

## 40% L1 norm prune unstructured


In [ ]:
conv_net = ConvNet()
conv_net.load_state_dict(torch.load(GLOBAL_PATH + 'Models/VGG_09.11.2022-16:50.pth', map_location=DEVICE))

In [ ]:
prune_l1_unstructured(conv_net, 0.4, 0)

In [ ]:
count_parameters(conv_net)

In [ ]:
test_model(conv_net, dl_test)

In [ ]:
train_func = partial(
    training_loop, n_epochs=2, loss_fn=torch.nn.CrossEntropyLoss(),
    dl_train=dl_train, dl_test=dl_test, device=DEVICE
)

In [ ]:
conv_net.to(DEVICE);

In [ ]:
optimizer = torch.optim.SGD(conv_net.parameters(), lr=0.05 * 6e-2, momentum=0.9, weight_decay=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.8)

In [ ]:
train_losses, test_losses, train_accs, test_accs = train_func(
    network=conv_net,
    optimizer=optimizer,
    scheduler=scheduler
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(train_accs, label="Точность на обучении", color='red', marker='.', linestyle='-.')
ax.plot(test_accs, label="Точность на тесте", color='red', marker='*')

ax.set_xlabel("Номер эпохи")
ax.set_ylabel("$\%$")

ax.grid(True)
ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
count_parameters(conv_net)

In [ ]:
test_model(conv_net, dl_test)

## 60% L1 norm prune unstructured


In [ ]:
conv_net = ConvNet()
conv_net.load_state_dict(torch.load(GLOBAL_PATH + 'Models/VGG_09.11.2022-16:50.pth', map_location=DEVICE))

In [ ]:
prune_l1_unstructured(conv_net, 0.6, 0)

In [ ]:
count_parameters(conv_net)

In [ ]:
test_model(conv_net, dl_test)

In [ ]:
train_func = partial(
    training_loop, n_epochs=2, loss_fn=torch.nn.CrossEntropyLoss(),
    dl_train=dl_train, dl_test=dl_test, device=DEVICE
)

In [ ]:
conv_net.to(DEVICE);

In [ ]:
optimizer = torch.optim.SGD(conv_net.parameters(), lr=0.05 * 4e-1, momentum=0.9, weight_decay=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.8)

In [ ]:
train_losses, test_losses, train_accs, test_accs = train_func(
    network=conv_net,
    optimizer=optimizer,
    scheduler=scheduler
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(train_accs, label="Точность на обучении", color='red', marker='.', linestyle='-.')
ax.plot(test_accs, label="Точность на тесте", color='red', marker='*')

ax.set_xlabel("Номер эпохи")
ax.set_ylabel("$\%$")

ax.grid(True)
ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
count_parameters(conv_net)

In [ ]:
test_model(conv_net, dl_test)